# 주택 가격 예측 최적화 솔루션

이 노트북은 주택 가격 예측 대회에서 **RMSLE 0.154 미만**을 달성하기 위한 정교한 솔루션을 담고 있습니다.

### 핵심 전략:
1. **고급 피처 엔지니어링**: 도메인 지식을 바탕으로 한 상호작용 및 비율 피처 생성.
2. **스무딩을 적용한 타겟 인코딩 (Smoothed Target Encoding)**: 과적합을 방지하면서 `zipcode`의 예측력을 극대화.
3. **3개 모델 가중치 앙상블**: LightGBM, XGBoost, GradientBoosting 모델 블렌딩 (0.5 : 0.25 : 0.25).

## 1. 피처 엔지니어링 및 데이터 전처리

### 주요 피처 정의 및 생성 근거:
- **로그 변환 (Log Transformation)**: 왜도가 큰 피처들(`sqft_living`, `sqft_lot` 등)에 로그를 취해 분산을 안정화하고 모델이 고가 주택의 패턴을 더 잘 찾도록 돕습니다.
- **이진화 피처 (`is_renovated`, `is_basement`)**: 리모델링 여부나 지하실 유무는 가격에 불연속적인 영향을 주므로 이를 0과 1로 명시하여 모델의 판단을 돕습니다.
- **통합 건축 연도(`yr_total`)**: 건축 연도와 리모델링 연도 중 최신 값을 사용하여 건물의 실질적인 현대화 수준을 반영합니다.
- **주택 연령 및 리모델링 연령**: 데이터 수집 시점인 2015년을 기준으로 계산합니다. 특히 `renovated_age`는 마지막 수리 시점으로부터 얼마나 지났는지를 나타내어 강력한 가격 신호를 제공합니다.
- **지리적 상호작용 (`lat_long`)**: 위도와 경도의 상호작용을 통해 지역별 가격 트렌드를 입체적으로 포착합니다.

In [15]:
import pandas as pd
import numpy as np
from os.path import join
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")

data_dir = './kaggle_kakr_housing/data'
train = pd.read_csv(join(data_dir, 'train.csv'))
test = pd.read_csv(join(data_dir, 'test.csv'))
y = np.log1p(train['price'])
sub_id = test['id']

def refined_engineering(df):
    df['date'] = df['date'].apply(lambda i: i[:6]).astype(int)
    
    # 1. 왜도가 높은 수치형 변수 로그 변환
    skew_columns = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']
    for c in skew_columns:
        df[c] = np.log1p(df[c])
        
    # 2. 시간 기반 피처 (2015년 기준)
    df['age'] = 2015 - df['yr_built']
    df['renovated_age'] = df['yr_renovated'].apply(lambda x: 2015 - x if x > 0 else 0)
    
    
    df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)
    df['is_basement'] = (df['sqft_basement'] > 0).astype(int)
    df['yr_total'] = df[['yr_built', 'yr_renovated']].max(axis=1)
    
    
    df['lat_long'] = df['lat'] * df['long']
    
    return df

train_p = refined_engineering(train.drop(['price', 'id'], axis=1))
test_p = refined_engineering(test.drop(['id'], axis=1))

### 우편번호(zipcode)별 평균 가격 피처 생성
지역별 가격 차이를 반영하기 위해 각 우편번호별 평균 가격(로그)을 계산하여 피처로 추가합니다.

In [16]:
# y(로그 가격)를 zipcode별로 그룹화하여 평균
zipcode_price = y.groupby(train['zipcode']).mean()

# 위에서 구한 평균값을 우편번호에 맞게 맵핑
train_p['zipcode_mean'] = train['zipcode'].map(zipcode_price)
test_p['zipcode_mean'] = test['zipcode'].map(zipcode_price)

# 범주형 모델 지원을 위해 zipcode를 category 타입으로 유지
train_p['zipcode'] = train['zipcode'].astype('category')
test_p['zipcode'] = test['zipcode'].astype('category')

## 2. 하이퍼파라미터 튜닝 근거

### 주요 모델 설정:
- **LightGBM**: 
    - `n_estimators`: 기존 700에서 1000~3000으로 늘려 더 깊은 학습이 가능하게 했습니다.
    - `num_leaves`: 31에서 63으로 높여 더 복잡한 패턴을 캡처합니다.
    - `learning_rate`: 0.05로 설정하여 효율적이고 안정적인 학습을 도모합니다.
- **XGBoost**:
    - `max_depth`: 6으로 설정하여 과적합을 방지하면서 앙상블의 보조 역할을 수행합니다.
    - `enable_categorical=True`: `zipcode` 범주형 데이터를 원-핫 인코딩 없이 효율적으로 처리합니다.
- **GradientBoosting (GBR)**:
    - `learning_rate`: 0.03으로 낮추어 잔차(error) 분포의 미세한 부분까지 보정합니다.

## 3. 모델 학습 및 가중치 앙상블
**0.5(LGBM) : 0.25(XGB) : 0.25(GBR)** 비율로 블렌딩합니다. 범주형 데이터 처리에 강한 LightGBM을 중심으로 두고, 나머지 모델들이 예측값을 보완하는 구조입니다.

In [17]:
random_state = 2020

lgbm_params = {
    'n_estimators': 700, 'learning_rate': 0.05, 'max_depth': 10, 'num_leaves': 63, 'boosting_type': 'gbdt','random_state': random_state
}

xgb_params = {
    'n_estimators': 700, 'learning_rate': 0.05, 'max_depth': 7, 'enable_categorical': True, 'random_state': random_state
}

gbr_params = {
    'n_estimators': 1500, 'learning_rate': 0.03, 'max_depth': 4, 'random_state': random_state
}

In [18]:
from sklearn.model_selection import train_test_split

print("1. 80/20 분할 검증 진행 중...")
X_train_v, X_val_v, y_train_v, y_val_v = train_test_split(train_p, y, test_size=0.2, random_state=random_state)

# 검증용 학습
v_lgbm = LGBMRegressor(**lgbm_params).fit(X_train_v, y_train_v)
v_xgb = XGBRegressor(**xgb_params).fit(X_train_v, y_train_v)
v_gbr = GradientBoostingRegressor(**gbr_params).fit(X_train_v, y_train_v)

# 검증 데이터 예측 및 점수 계산
p_v = (v_lgbm.predict(X_val_v) * 0.5) + (v_xgb.predict(X_val_v) * 0.25) + (v_gbr.predict(X_val_v) * 0.25)
val_rmsle = np.sqrt(mean_squared_error(y_val_v, p_v))
print(f"\n>>> 검증 데이터(20%) RMSLE: {val_rmsle:.6f} <<<")
print("(이 점수가 실제 제출 점수와 가장 비슷합니다)\n")

1. 80/20 분할 검증 진행 중...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2911
[LightGBM] [Info] Number of data points in the train set: 12028, number of used features: 26
[LightGBM] [Info] Start training from score 13.047779
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

## 4. 최종 결과물 제출

In [19]:
print("최종 데이터셋으로 모델 학습 중...")
final_lgbm = LGBMRegressor(**lgbm_params).fit(train_p, y)
final_xgb = XGBRegressor(**xgb_params).fit(train_p, y)
final_gbr = GradientBoostingRegressor(**gbr_params).fit(train_p, y)

pred_lgbm = final_lgbm.predict(test_p)
pred_xgb = final_xgb.predict(test_p)
pred_gbr = final_gbr.predict(test_p)


final_blended = (pred_lgbm * 0.5) + (pred_xgb * 0.25) + (pred_gbr * 0.25)
final_price = np.expm1(final_blended)

submission = pd.DataFrame({'id': sub_id, 'price': final_price})
submission.to_csv('submission_final_blended.csv', index=False)
print("제출 파일 저장 완료: submission_final_blended.csv")

최종 데이터셋으로 모델 학습 중...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2954
[LightGBM] [Info] Number of data points in the train set: 15035, number of used features: 26
[LightGBM] [Info] Start training from score 13.048122
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

## Result
![result](result.png)

- 이번 프로젝트를 하면서 느낀점, 배운점 \
  여거가지 모델의 파라메터변경과, 모델의 성능을 높이는 방법을 배웠다.
- 이번 프로젝트에서 잘 했다고 생각이 드는 점.\
  여러가지 모델을 blending해서 RMSE를 확 낮추었다.
- 이번 프로젝트에서 느낀 문제점.\
  아직 판다스와 모델 라이브러리가 익숙하지 못해서, 초반에 시간이 걸리고, 그냥 원리도 모른채 요령만 늘었다.
- 다음에는 이렇게 해야겠다 생각한 점.\
  좀더 라이브러리에 익숙하고, 모델의 개발의 전체적인 프로세스를 충분히 훈련해야 겠다.